# Dataset Cleanup

In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('data/train.csv', index_col='PassengerId')
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [22]:
df.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [23]:
def prepare_data(df):
    df.drop(['Name', 'Ticket'], axis=1, inplace=True)

    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Fare'].fillna(df['Fare'].mean(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Cabin'].fillna('U', inplace=True)
    df['Cabin'] = df['Cabin'].apply(lambda x: x.split()[0][0])

    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
    categorized_age = pd.cut(df['Age'], bins)
    df['Categorized_age'] = categorized_age
    df = df.drop(['Age'], axis=1)
    
    cagegorized_age_columns = pd.get_dummies(df['Categorized_age'], prefix='Categorized_age')
    df = pd.concat([df, cagegorized_age_columns], axis=1)
    df = df.drop(['Categorized_age'], axis=1)

    cabin_columns = pd.get_dummies(df['Cabin'], prefix='Desk')
    df = pd.concat([df, cabin_columns], axis=1)
    df.drop('Cabin', axis=1, inplace=True)
    if 'Desk_T' not in df.columns:
        df.insert(21, 'Desk_T', False)

    gender_columns = pd.get_dummies(df['Sex'], prefix='Sex')
    df = pd.concat([df, gender_columns], axis=1)
    df.drop('Sex', axis=1, inplace=True)

    embarked_columns = pd.get_dummies(df['Embarked'], prefix='Embarked')
    df = pd.concat([df, embarked_columns], axis=1)
    df.drop('Embarked', axis=1, inplace=True)

    pclass_columns = pd.get_dummies(df['Pclass'], prefix='Pclass')
    df = pd.concat([df, pclass_columns], axis=1)
    df.drop('Pclass', axis=1, inplace=True)
    
    return df

In [24]:
df = prepare_data(df)
df.head()

,Survived,SibSp,Parch,Fare,"Categorized_age_(0, 10]","Categorized_age_(10, 20]","Categorized_age_(20, 30]","Categorized_age_(30, 40]","Categorized_age_(40, 50]","Categorized_age_(50, 60]",...,Desk_T,Desk_U,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,7.2500,False,False,True,False,False,False,...,False,True,False,True,False,False,True,False,False,True
2,1,1,0,71.2833,False,False,False,True,False,False,...,False,False,True,False,True,False,False,True,False,False
3,1,0,0,7.9250,False,False,True,False,False,False,...,False,True,True,False,False,False,True,False,False,True
4,1,1,0,53.1000,False,False,False,True,False,False,...,False,False,True,False,False,False,True,True,False,False
5,0,0,0,8.0500,False,False,False,True,False,False,...,False,True,False,True,False,False,True,False,False,True


In [25]:
df.to_csv('data/train_clean.csv')

In [26]:
test_df = pd.read_csv('data/test.csv', index_col='PassengerId')
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [27]:
test_df.isna().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [28]:
test_df = prepare_data(test_df)
test_df.head()

,SibSp,Parch,Fare,"Categorized_age_(0, 10]","Categorized_age_(10, 20]","Categorized_age_(20, 30]","Categorized_age_(30, 40]","Categorized_age_(40, 50]","Categorized_age_(50, 60]","Categorized_age_(60, 70]",...,Desk_T,Desk_U,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,0,0,7.8292,False,False,False,True,False,False,False,...,False,True,False,True,False,True,False,False,False,True
893,1,0,7.0000,False,False,False,False,True,False,False,...,False,True,True,False,False,False,True,False,False,True
894,0,0,9.6875,False,False,False,False,False,False,True,...,False,True,False,True,False,True,False,False,True,False
895,0,0,8.6625,False,False,True,False,False,False,False,...,False,True,False,True,False,False,True,False,False,True
896,1,1,12.2875,False,False,True,False,False,False,False,...,False,True,True,False,False,False,True,False,False,True


In [29]:
test_df.to_csv('data/test_clean.csv')